In [8]:
import os

In [9]:
from langchain_community.document_loaders import PyMuPDFLoader

In [10]:
loader = PyMuPDFLoader("industry_laws.pdf")
pages = loader.load()

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
# 문서를 문장으로 분리
## 청크 크기 500, 각 청크의 50자씩 겹치도록 청크를 나눈다
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)
docs = text_splitter.split_documents(pages)

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
# 문장을 임베딩으로 변환하고 벡터 저장소에 저장
embeddings = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings':True},
)

In [15]:
# 벡터 저장소 생성
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(docs, embeddings)


# 벡터 저장소 경로 설정
## 현재 경로에 'vectorstore' 경로 생성
vectorstore_path = 'vectorstore'
os.makedirs(vectorstore_path, exist_ok=True)

# 벡터 저장소 생성 및 저장
vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=vectorstore_path)
# 벡터스토어 데이터를 디스크에 저장
vectorstore.persist()
print("Vectorstore created and persisted")

Vectorstore created and persisted


/tmp/ipykernel_30922/157380280.py:14: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [16]:
from langchain_community.chat_models import ChatOllama

# Ollama 를 이용해 로컬에서 LLM 실행
## llama3-ko-instruct 모델 다운로드는 Ollama 사용법 참조
model = ChatOllama(model="llama3.1", temperature=0)

/tmp/ipykernel_30922/1082334087.py:5: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(model="llama3.1", temperature=0)


In [17]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 5})

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


# Prompt 템플릿 생성
template = '''친절한 챗봇으로서 상대방의 요청에 최대한 자세하고 친절하게 답하자. 모든 대답은 한국어(Korean)으로 대답해줘.":
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

Query: 연장근로수당에 대해 알려 줘
Answer: 연장근로수당에 대한 정보를 알려드리겠습니다.

연장근로수당은 근로자가 정상근무시간을 초과하여 근무하는 경우, 그에 따른 보상을 받는 것입니다. 이 수당은 노동법에서 규정하고 있으며, 사업주가 근로자에게 지급해야 하는 금액입니다.

연장근로는 다음과 같은 경우에 해당합니다.

1.  정상근무시간을 초과하여 근무하는 경우
2.  휴일 또는 공휴일에 근무하는 경우
3.  야간근무를 하는 경우

연장근로수당의 지급기준은 다음과 같습니다.

*   연장근로수당의 지급률은 노동부에서 정한 기준에 따라 결정됩니다.
*   연장근로는 1시간 이상 근무하는 경우에만 지급됩니다.
*   연장근로수당은 근로자의 월평균임금의 25%를 초과하지 않는 범위 내에서 지급됩니다.

연장근로수당을 받으려면, 사업주가 근로자에게 다음과 같은 사항을 알려줘야 합니다.

1.  연장근로의 시작 및 종료 시간
2.  연장근로의 내용 및 목적
3.  연장근로에 대한 보상금액

사업주는 근로자가 정상근무시간을 초과하여 근무하는 경우, 그에 따른 보상을 받도록 하여야 합니다. 또한, 사업주가 근로자에게 연장근로에 대한 정보를 제공해야 하며, 연장근로수당의 지급기준을 준수해야 합니다.

궁금한 점이 있으시면 언제든지 질문해 주세요!


In [ ]:
# Chain 실행
query = "연장근로수당에 대해 알려 줘"
answer = rag_chain.invoke(query)

print("Query:", query)
print("Answer:", answer)